In [143]:
!pip install gensim
import gensim.downloader as api
import numpy as np
import random
import os
from IPython.display import clear_output
import time

In [6]:
def similaridade_cosseno(vector1,vector2):
  return np.dot(vector1,vector2)/ np.sqrt(np.dot(vector1,vector1) * np.dot(vector2,vector2))

## Instanciando o Embedder de Palavras

Usaremos a biblioteca gensim, mais especificamente o modelo **glove-wiki-gigaword-100** para transformar as palavras em embeddings.
Cada palavra se transforma em um vetor de 100 dimensões

In [8]:
word_embedder = api.load("glove-wiki-gigaword-100")

In [10]:
teste = word_embedder["man"]
print(teste[:15])
print(len(teste))

[ 0.37293    0.38503    0.71086   -0.65911   -0.0010128  0.92715
  0.27615   -0.056203  -0.24294    0.24632   -0.18449    0.31398
  0.48983    0.09256    0.32958  ]
100


Os word embeddings conservam algumas estruturas que fazem sentido lógico para os seres humanos, mantendo valor e relação entre palavras mesmo com poucas dimensões. Por exemplo, a seguir é visto algo lógico:

> homem + negocio = homem de negocio

Vejamos se os embeddings são de fato similares, com uma similaridade de 0.48, não exatamente baixa, especialmente para esse nível de lógica

In [14]:
homem = word_embedder["man"]
negocio = word_embedder["sales"]
homem_de_negocio = word_embedder["salesman"]

similaridade_cosseno(homem + negocio, homem_de_negocio)

np.float32(0.48196864)

Note que o mesmo não pode ser dito para exemplos sem sentido como

> Pessoa + gato = sorvete

Gerando uma similaridade bastante baixa

In [17]:
pessoa = word_embedder["person"]
gato = word_embedder["cat"]
sorvete = word_embedder["icecream"]

similaridade_cosseno(pessoa + gato, sorvete)

np.float32(-0.20359729)

Podemos, para fins de entendimento, criar uma função, que recebe essas duas palavras e acha a junção mais provável e lógica possível verificando as similaridades.

In [71]:
def get_palavras_similares(palavra1, palavra2, topn=20,threshold = 0.6):

    results = word_embedder.most_similar(
        positive=[palavra1,palavra2],
        topn=topn
    )
    filtered = []
    for result in results:
      if result[1] >= threshold:
        filtered.append(result)
    return filtered

In [72]:
get_palavras_similares("sales","man")

[('one', 0.7892695069313049),
 ('same', 0.7755798697471619),
 ('another', 0.7643623352050781),
 ('last', 0.7641332149505615),
 ('seen', 0.7624484896659851),
 ('but', 0.7561348080635071),
 ('even', 0.7556900382041931),
 ('than', 0.7533072829246521),
 ('as', 0.7526919841766357),
 ('year', 0.7507821321487427),
 ('while', 0.7507364749908447),
 ('though', 0.7500261664390564),
 ('now', 0.7497562766075134),
 ('turned', 0.7468592524528503),
 ('making', 0.7459621429443359),
 ('whose', 0.7447161078453064),
 ('just', 0.7439008951187134),
 ('that', 0.7421001195907593),
 ('only', 0.7415590882301331),
 ('well', 0.7384680509567261)]

Note, no entanto, que nem sempre há um sentido tão forte, isso por que existem muitas palavras em textos que acompanham "sales" e "man", não somente salesman.

In [78]:
get_palavras_similares("computer","scientist")

[('technology', 0.7423041462898254),
 ('science', 0.7107239365577698),
 ('researcher', 0.6956336498260498),
 ('expert', 0.6914530992507935),
 ('lab', 0.6911506652832031),
 ('research', 0.6774166822433472),
 ('software', 0.67385333776474),
 ('laboratory', 0.672516942024231),
 ('professor', 0.6526342630386353),
 ('computers', 0.6489235162734985),
 ('scientists', 0.6458507180213928),
 ('programmer', 0.6323270201683044),
 ('engineering', 0.6316056251525879),
 ('physicist', 0.6289891600608826),
 ('researchers', 0.6215561628341675),
 ('computing', 0.6201502084732056),
 ('engineer', 0.6173214912414551),
 ('institute', 0.6079432964324951),
 ('master', 0.6067216396331787),
 ('mit', 0.6004737615585327)]

Porém, pode observar que palavras como "computer" e "scientist" aparece palavras mais comuns logicamente como "programmer".

## Jogo Interativo

Com base nesses principios, um jogo será feito onde o objetivo é o usuário tentar acertar uma palavra que una duas outras em 3 categorias definidas:


1.   Objetos
2.   Profissões
3.   Comidas

O jogo funciona da seguinte forma:



1.   Uma lista definida de palavras é feita para cada categoria
2.   São escolhidas uma categoria aleatoriamente a cada rodada
3.   Duas palavras são escolhidas caso houver uma palavra que resulte da soma dessas duas com uma similaridade de cosseno maior ou igual a 0.6
4.  O usuário deve escrever uma palavra
5.  Se essa palavra tiver uma similaridade maior ou igual a 0.6 acerta e vai para a próxima rodada
6.  Caso contrário perde uma vida
7.  O usuário pode, uma única vez, pedir ajuda para os jurados
8.  Os jurados tem 50% de chance de dizer uma palavra correta, que é basicamente uma busca aleatória entre as top_n palavras similares da junção das duas palavras anteriores
9.  Caso cair no outro 50% de probabilidade, os jurados podem simplesmente dizer uma palavra aleatória (que pode raramente, realmente ser similar, mas é um tiro no escuro).


O sistema de pontuação e vida é simples



*   3 vidas no total, com 1 perda por erro
*   Um score que aumenta de acordo com a similaridade da palavra
$$
\text{similaridade de cosseno} \times 1000
$$






### Palavras das 3 categorias

In [228]:
OBJETOS = [
    "chair", "table", "phone", "computer", "keyboard", "screen",
    "lamp", "door", "window", "car", "bottle", "clock",
    "pen", "pencil", "notebook", "book", "paper", "backpack",
    "wallet", "glasses", "mirror", "bed", "sofa", "couch",
    "television", "remote", "radio", "camera", "microphone",
    "speaker", "headphone", "watch", "ring", "bracelet",
    "cup", "mug", "plate", "fork", "knife", "spoon",
    "pan", "pot", "refrigerator", "oven", "stove", "microwave",
    "toaster", "blender", "fan", "heater", "airplane",
    "bicycle", "motorcycle", "truck", "bus", "train",
    "boat", "ship", "helmet", "umbrella", "pillow",
    "blanket", "towel", "soap", "shampoo", "toothbrush",
    "toothpaste", "bag", "box", "basket", "key",
    "lock", "ladder", "tool", "hammer", "screwdriver",
    "drill", "rope", "chain", "battery", "flashlight",
    "candle", "lighter", "map", "compass", "ticket",
    "card", "passport", "scissors", "ruler", "eraser"
]

COMIDAS = [
    "bread", "cheese", "coffee", "tea", "milk", "rice",
    "pizza", "burger", "cake", "soup", "chicken", "apple",
    "banana", "orange", "grape", "pear", "peach", "mango",
    "watermelon", "strawberry", "blueberry", "pineapple",
    "carrot", "potato", "onion", "garlic", "tomato",
    "lettuce", "cabbage", "spinach", "broccoli", "corn",
    "beans", "lentils", "pasta", "noodles", "steak",
    "fish", "salmon", "tuna", "shrimp", "egg",
    "omelet", "sausage", "bacon", "ham", "sandwich",
    "hotdog", "fries", "chocolate", "cookie", "icecream",
    "cereal", "yogurt", "butter", "cream", "honey",
    "jam", "sugar", "salt", "pepper", "spice",
    "sushi", "taco", "burrito", "pancake", "waffle",
    "donut", "pie", "lasagna", "risotto", "curry",
    "sauce", "ketchup", "mustard", "mayonnaise", "salad",
    "juice", "water", "soda", "wine", "beer"
]
PROFISSOES = [
    "doctor", "nurse", "teacher", "engineer", "lawyer",
    "chef", "driver", "pilot", "farmer", "artist",
    "police", "firefighter", "soldier", "dentist",
    "pharmacist", "scientist", "researcher", "writer",
    "journalist", "actor", "singer", "musician",
    "dancer", "photographer", "designer", "architect",
    "programmer", "developer", "analyst", "manager",
    "director", "accountant", "banker", "clerk",
    "cashier", "waiter", "bartender", "mechanic",
    "electrician", "plumber", "carpenter", "builder",
    "coach", "trainer", "athlete", "judge",
    "politician", "consultant", "therapist", "psychologist",
    "professor", "librarian", "translator", "editor",
    "technician", "guard", "security", "surgeon",
    "veterinarian", "astronaut", "entrepreneur",
    "marketer", "salesman", "receptionist", "cleaner"
]

### Funções para achar palavras semelhantes

Pega um conjunto de palavras anteriores para achar as semelhantes. Após isso, a pergunta é formulada com base nessas palavras e retorna o status do usuário (Se respondeu ou pediu ajuda aos jurados).

In [154]:

def achar_conjunto_palavra_semelhante(
    word_embedder,
    min_sim=0.60,
    topn=50,
    base=[],
    max_tentativas=10_000
):


    base = [w for w in base if w in word_embedder]


    for _ in range(max_tentativas):
        w1, w2 = random.sample(base, 2)

        candidatos = word_embedder.most_similar(
            positive=[w1, w2],
            topn=topn
        )

        for w3, sim in candidatos:
            if w3 in {w1, w2}:
                continue
            if w3 in base and sim >= min_sim:
                return (w1, w2, w3, float(sim))

    return None

def pergunta(score, word_embedder, tem_jurados=False, min_sim=0.60):
    categoria = ""
    random_var = random.randint(0,3)
    if random_var == 0:
        base = OBJETOS
        categoria = "Objetos"
    elif random_var == 1:
        base = COMIDAS
        categoria = "Comidas"
    else:
        base = PROFISSOES
        categoria = "Profissões"
    trio = achar_conjunto_palavra_semelhante(
        word_embedder,
        base=base,
        min_sim=min_sim
    )

    w1, w2, w3, sim = trio

    largura = 80
    print("╔" + "═" * largura + "╗")
    titulo = "  RODADA: ADIVINHE A PALAVRA  "
    print("║" + titulo.center(largura) + "║")
    print("╠" + "═" * largura + "╣")

    linha_info = f"Score: {score}"
    linha_cat = f"Categoria: {categoria}"
    print("║" + (linha_cat).ljust(largura - len(linha_info)) + linha_info + "║")

    print("║" + " " * largura + "║")
    conta = f"{w1}  +  {w2}  =  ?"
    print("║" + conta.center(largura) + "║")
    print("║" + " " * largura + "║")

    print("╠" + "═" * largura + "╣")
    if tem_jurados:
        print("║ " + "Digite sua resposta ou 'J' para chamar os jurados 🎤".ljust(largura - 1) + "║")
    else:
        print("║ " + "Digite sua resposta:".ljust(largura - 1) + "║")
    print("╚" + "═" * largura + "╝")

    resp = input("> ").strip().lower()

    if tem_jurados and resp in {"j"}:
        return {
            "status": "jurados",
            "resposta": None,
            "w1": w1, "w2": w2, "w3": w3, "sim": sim,
            "score": score
        }

    return {
        "status": "respondeu",
        "resposta": resp,
        "w1": w1, "w2": w2, "w3": w3, "sim": sim,
        "score": score
    }

### Menu e escolhas

Aqui onde as respostas do usuário são verificadas e o sistema é atualizado.

In [229]:
def menu_inicial():
    largura = 80
    print("\n" + "╔" + "═" * largura + "╗")
    print("║" + " " * largura + "║")
    print("║" + "  BEM-VINDO AO JOGO DE ADIVINHAÇÃO SEMÂNTICA  ".center(largura) + "║")
    print("║" + " " * largura + "║")
    print("╠" + "═" * largura + "╣")
    print("║ REGRAS DO JOGO:".ljust(largura) + "║")
    print("║".ljust(largura + 1) + "║")
    print("║ 1.  Você deve adivinhar uma palavra que se assemelhe".ljust(largura) + "║")
    print("║     à junção das duas palavras fornecidas.".ljust(largura) + "║")
    print("║".ljust(largura + 1) + "║")
    print("║ 2.  Se o Computertron 🤖 detectar similaridade ≥ 60%,".ljust(largura) + "║")
    print("║     você avança para a próxima etapa.".ljust(largura) + "║")
    print("║".ljust(largura + 1) + "║")
    print("║ 3.  Você tem 3 vidas no total.".ljust(largura) + "║")
    print("║     Cada erro custa uma vida.".ljust(largura) + "║")
    print("║".ljust(largura + 1) + "║")
    print("║ 4.  Pode pedir ajuda aos jurados 🎤".ljust(largura) + "║")
    print("║     (50% de chance de receber uma dica útil...".ljust(largura) + "║")
    print("║     Mas será que você confia nas probabilidades?)".ljust(largura) + "║")
    print("║".ljust(largura + 1) + "║")
    print("╠" + "═" * largura + "╣")
    print("║" + "  ESTÁ PRONTO?!?!?!!!!  ".center(largura) + "║")
    print("╚" + "═" * largura + "╝\n")
    print("1. Sim")
    print("2. Sair")
    if int(input()) == 1:
      return True
    else:
      return False


def ajuda_jurados(resposta):
          print("JURADOS AJUDEM!!!")
          tem_jurados = False
          time.sleep(2)
          random_chance = random.randint(0,100)
          if random_chance <= 50:
            print("Você recebeu uma dica!")
            palavra_similar_aleatoria = random.choice(get_palavras_similares(resposta["w1"],resposta["w2"]))
            print("Achamos que é: ", palavra_similar_aleatoria[0])
            time.sleep(2)
            resp = input("> ").strip().lower()

            return {
            "status": "respondeu",
            "resposta": resp,
            "w1": resposta["w1"], "w2": resposta["w2"], "w3": resposta["w3"],
            }
          else:
            print("Achamos que é: ", random.choice(word_embedder.index_to_key))
            time.sleep(2)
            resp = input("> ").strip().lower()
            return {
            "status": "respondeu",
            "resposta": resp,
            "w1": resposta["w1"], "w2": resposta["w2"], "w3": resposta["w3"],
            }


def verificar_resposta(similaridade_cosseno,palavra1,palavra2,palavra_resposta):
  if palavra_resposta == palavra1 or palavra_resposta == palavra2:
    return False
  else:
    if similaridade_cosseno >= 0.6:
      return True
    else:
      return False


def limpar_console():
    clear_output(wait=True)



In [230]:
def jogo_adivinhacao():
    iniciar = menu_inicial()
    if iniciar:
      tem_jurados = True
      score = 0
      vidas = 3
      while vidas > 0:
        limpar_console()
        time.sleep(2)
        resposta = pergunta(score, word_embedder, tem_jurados=tem_jurados, min_sim=0.60)
        if resposta["status"] == "respondeu":
          try:
            similaridade_resposta = similaridade_cosseno(word_embedder[resposta["resposta"]], word_embedder[resposta["w1"]] + word_embedder[resposta["w2"]])
          except KeyError:
              print("Errou!")
              vidas -= 1
              print(f"Você tem {vidas} vidas restantes")
          if verificar_resposta(similaridade_resposta,resposta["w1"],resposta["w2"],resposta["resposta"]):
            pontos = int(similaridade_resposta * 1000)
            score += pontos
            print(f"E Acertou! \033[92m+{pontos}\033[0m")
          else:
            print("Errou!")
            vidas -= 1
            print(f"Você tem {vidas} vidas restantes")
        elif resposta["status"] == "jurados" and tem_jurados:
          resposta_jurados = ajuda_jurados(resposta)
          if resposta_jurados["status"] == "respondeu":
            tem_jurados = False
            try:
              similaridade_resposta = similaridade_cosseno(word_embedder[resposta_jurados["resposta"]], word_embedder[resposta_jurados["w1"]] + word_embedder[resposta_jurados["w2"]])
            except KeyError:
              print("Errou!")
              vidas -= 1
              print(f"Você tem {vidas} vidas restantes")
            if verificar_resposta(similaridade_resposta,resposta_jurados["w1"],resposta_jurados["w2"],resposta_jurados["resposta"]):
              pontos = int(similaridade_resposta * 1000)
              score += pontos
              print(f"E Acertou! \033[92m+{pontos}\033[0m")
            else:
              print("Errou!")
              vidas -= 1
              if vidas > 0:
                print(f"Você tem {vidas} vidas restantes")
              else:
                print(f"Fim de jogo! Mas olha só, você ganhou ${score} dólares!")







In [232]:
jogo_adivinhacao()


╔════════════════════════════════════════════════════════════════════════════════╗
║                                                                                ║
║                   BEM-VINDO AO JOGO DE ADIVINHAÇÃO SEMÂNTICA                   ║
║                                                                                ║
╠════════════════════════════════════════════════════════════════════════════════╣
║ REGRAS DO JOGO:                                                               ║
║                                                                                ║
║ 1.  Você deve adivinhar uma palavra que se assemelhe                          ║
║     à junção das duas palavras fornecidas.                                    ║
║                                                                                ║
║ 2.  Se o Computertron 🤖 detectar similaridade ≥ 60%,                          ║
║     você avança para a próxima etapa.                                         ║
║       

KeyboardInterrupt: Interrupted by user